<a href="https://colab.research.google.com/github/lucymccanna/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [813 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
# load in data
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Books_v1_02.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

In [4]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ58W7SMO911M|          4|            2|          3|   N|                N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|                N|
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
| RATOTLA3OF70O|          5|            2|          2|   N|                N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [5]:
# #1 Filter the data and create a new DataFrame or table to retrieve all the rows where 
# the total_votes count is equal to or greater than 20 to pick reviews that are more 
# likely to be helpful and to avoid having division by zero errors later on.
vine_totalvotesfilter_df = vine_df.filter("total_votes>19")
vine_totalvotesfilter_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1AABFZH0J0C0C|          3|           17|         35|   N|                N|
|R3SJYLG07EHNE9|          2|           16|         34|   N|                N|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
|R1KPOIT5WT3NEF|          1|           13|         29|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
| R33IFUWCNYFSO|          2|            4|         45|   N|                N|
| RWMT2MXG3WLDP|          5|            6|         20|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|     

In [6]:
# pyspark not recognizing 'col' - workaround: https://stackoverflow.com/questions/40163106/cannot-find-col-function-in-pyspark
from pyspark.sql import functions as F

In [7]:
vine_totalvotesfilter_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



In [8]:
#2 Filter the new DataFrame or table created in Step 1 and create a new DataFrame or 
# table to retrieve all the rows where the number of helpful_votes divided by 
# total_votes is equal to or greater than 50%.

#add column
vine_percentadded_df = vine_totalvotesfilter_df.withColumn("percent help/tot", F.col('helpful_votes')/F.col('total_votes')* 100)
vine_percentadded_df.show()



+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|  percent help/tot|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1DOSHH6AI622S|          4|            1|         22|   N|                N| 4.545454545454546|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|              80.0|
|R1AABFZH0J0C0C|          3|           17|         35|   N|                N| 48.57142857142857|
|R3SJYLG07EHNE9|          2|           16|         34|   N|                N| 47.05882352941176|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N| 67.74193548387096|
|R1KPOIT5WT3NEF|          1|           13|         29|   N|                N|44.827586206896555|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N| 96.42857142857143|
| R33IFUWCNYFSO|          2|  

In [9]:
# filter for greater than or equal to 50% 
# filter for equal greater than 50% 
vine_percentfilteradded_df = vine_percentadded_df.filter(F.col("percent help/tot") >= 50)
vine_percentfilteradded_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase| percent help/tot|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|             80.0|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|67.74193548387096|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|96.42857142857143|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|            100.0|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|87.29641693811075|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|88.88888888888889|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|95.83333333333334|
|R2JSWNTAO83X3V|          5|          14

In [10]:
# 3 Create a new DataFrame or table that retrieves all the rows where a review was written as part of the 
# Vine program (paid), vine == 'Y'

vine_paidreviews_df = vine_percentfilteradded_df.filter(F.col("vine")=='Y')
vine_paidreviews_df.show()


+---------+-----------+-------------+-----------+----+-----------------+----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percent help/tot|
+---------+-----------+-------------+-----------+----+-----------------+----------------+
+---------+-----------+-------------+-----------+----+-----------------+----------------+



In [11]:
# 4 retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
vine_unpaidreviews_df = vine_percentfilteradded_df.filter(F.col("vine")=='N')
vine_unpaidreviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase| percent help/tot|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|             80.0|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|67.74193548387096|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|96.42857142857143|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|            100.0|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|87.29641693811075|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|88.88888888888889|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|95.83333333333334|
|R2JSWNTAO83X3V|          5|          14

In [30]:
#5 Determine the total number of reviews, the number of 5-star reviews, 
# and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

# Unpaid
# https://stackoverflow.com/questions/47812526/pyspark-sum-a-column-in-dataframe-and-return-results-as-int
# Percentage of 5 star reviews
vine_unpaidstats_df = vine_unpaidreviews_df.select("vine", "star_rating")
vine_unpaidstats_df.show()


+----+-----------+
|vine|star_rating|
+----+-----------+
|   N|          5|
|   N|          5|
|   N|          5|
|   N|          4|
|   N|          1|
|   N|          4|
|   N|          5|
|   N|          5|
|   N|          5|
|   N|          4|
|   N|          2|
|   N|          5|
|   N|          4|
|   N|          4|
|   N|          5|
|   N|          5|
|   N|          5|
|   N|          4|
|   N|          5|
|   N|          1|
+----+-----------+
only showing top 20 rows



In [50]:
# Total number of reviews
total_number_reviews = vine_percentfilteradded_df.count()
total_number_reviews

403807

In [49]:
# Calculate number of unpaid reviews
number_reviews_unpaid = vine_unpaidstats_df.count()
number_reviews_unpaid

403807

In [40]:
number_fivestar_reviews_df = vine_unpaidstats_df.filter("star_rating==5")
number_fivestar_reviews_df.show()

+----+-----------+
|vine|star_rating|
+----+-----------+
|   N|          5|
|   N|          5|
|   N|          5|
|   N|          5|
|   N|          5|
|   N|          5|
|   N|          5|
|   N|          5|
|   N|          5|
|   N|          5|
|   N|          5|
|   N|          5|
|   N|          5|
|   N|          5|
|   N|          5|
|   N|          5|
|   N|          5|
|   N|          5|
|   N|          5|
|   N|          5|
+----+-----------+
only showing top 20 rows



In [51]:
number_fivestar_reviews_unpaid = number_fivestar_reviews_df.count()
number_fivestar_reviews_unpaid

242889

In [46]:
percentage_fivestar_unpaid = (number_fivestar_reviews_unpaid/number_reviews_unpaid) * 100
percentage_fivestar_unpaid

60.14977452099642

In [48]:
# Paid reviews
number_reviews_paid = vine_paidreviews_df.count()
number_reviews_paid

0